# Final Assignment_battle of neighborhoods

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

## 1. Get the data of Chinese-Americans population in each city

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_U.S._cities_with_significant_Chinese-American_populations'
match = 'State'
dfs = pd.read_html(url, match = match)

### Clean up the data

In [3]:
dfs[0]

,Rank,City,State,Chinese-Americans,Percentage
0,1,San Francisco[n 1],California,180372,21.2
1,2,Irvine,California,34022,13.8
2,3,Oakland,California,34106,8.3
3,4,San Jose,California,75582,7.5
4,5,New York[n 2],New York,562205,6.6
5,6,Plano,Texas,14822,5.3
6,7,Honolulu[n 3],Hawaii,51149,5.2
7,8,Boston,Massachusetts,30097,4.6
8,9,Seattle,Washington,30223,4.5
9,10,Sacramento,California,21691,4.5


In [5]:
dfs[0].at[0,'City'] = 'San Francisco'
dfs[0].at[4,'City'] = 'New York'
dfs[0].at[6,'City'] = 'Honolulu'
dfs[0].at[12,'City'] = 'Fairfax'
dfs[0].at[13,'City'] = 'Philadelphia'
dfs[0].at[27,'City'] = 'Lexington'
dfs[0].at[28,'City'] = 'Washington'
dfs[0].head(10)

,Rank,City,State,Chinese-Americans,Percentage
0,1,San Francisco,California,180372,21.2
1,2,Irvine,California,34022,13.8
2,3,Oakland,California,34106,8.3
3,4,San Jose,California,75582,7.5
4,5,New York,New York,562205,6.6
5,6,Plano,Texas,14822,5.3
6,7,Honolulu,Hawaii,51149,5.2
7,8,Boston,Massachusetts,30097,4.6
8,9,Seattle,Washington,30223,4.5
9,10,Sacramento,California,21691,4.5


In [6]:
dfs[2]

2010-2016                                                               
        Rank              City           State Chinese-Americans Percentage
0          1         Daly City      California             20048       19.0
1          2           Fremont      California             40752       17.9
2          3          El Monte      California             20190       17.5
3          4            Irvine      California             34022       13.8
4          5         Sunnyvale      California             19395       13.0
5          6          Bellevue      Washington             15804       11.6
6          7       West Covina      California             12193       11.3
7          8          Berkeley      California             10094        8.5
8          9         Elk Grove      California             10758        6.6
9         10         Cambridge   Massachusetts              7404        6.8
10        11          Pasadena      California              8867        6.3
11        12         Ann Arbor        Michigan              7998        6.8
12        13          Torrance      California              7696        5.2
13        14     Spring Valley          Nevada             10927        5.7
14        15          Richmond      California              5523        5.1
15        16        Enterprise          Nevada              5879        4.5
16        17           Hayward      California              6930        4.5
17        18        Naperville        Illinois              6584        4.5
18        19         Fullerton      California              5328        3.8
19        20        Richardson           Texas              4646        4.3
20        21   College Station           Texas              4079        3.9
21        22     Thousand Oaks      California              3889        3.0
22        23  Rancho Cucamonga      California              5778        3.3
23        24              Cary  North Carolina              5283        3.4
24        25            Pomona      California              4867        3.2
25        26       Jersey City      New Jersey              8420        3.2
26        27           Concord      California              3607        2.8
27        28           Madison       Wisconsin              6899        2.8
28        29             Tempe         Arizona              4361        2.5
29        30         New Haven     Connecticut              2635        2.0
30        31              Kent      Washington              3273        2.6
31        32           Antioch      California              2342        2.2
32        33          Chandler         Arizona              5273        2.2
33        34            Orange      California              2729        2.0
34        35          Columbia        Missouri              2765        2.4
35        36    Arlington[n 8]        Virginia              4827        2.1
36        37  Huntington Beach      California              3925        2.0
37        38           Boulder        Colorado              2376        2.3
38        39       Gainesville         Florida              2358        1.8
39        40          Glendale      California              3818        1.9
40        41            Durham  North Carolina              3990        1.6
41        42      Garden Grove      California              3032        1.7
42        43          Stamford     Connecticut              1884        1.5
43        44    Salt Lake City            Utah              3037        1.6
44        45            Eugene          Oregon              2487        1.6
45        46          Syracuse        New York              2351        1.6
46        47           Norwalk      California              1719        1.6
47        48        Providence    Rhode Island              2647        1.5
48        49            Frisco           Texas              2077        1.5
49        50           Miramar         Florida              1993        1.5
50        51            Norman        Oklahoma     

In [7]:
dfs[2].columns = ['Rank', 'City', 'State', 'Chinese-Americans', 'Percentage']
dfs[2].at[35, 'City'] = 'Arlington'
dfs[2].drop(index=3, inplace = True)
dfs[2].head(10)

,Rank,City,State,Chinese-Americans,Percentage
0,1,Daly City,California,20048,19.0
1,2,Fremont,California,40752,17.9
2,3,El Monte,California,20190,17.5
4,5,Sunnyvale,California,19395,13.0
5,6,Bellevue,Washington,15804,11.6
6,7,West Covina,California,12193,11.3
7,8,Berkeley,California,10094,8.5
8,9,Elk Grove,California,10758,6.6
9,10,Cambridge,Massachusetts,7404,6.8
10,11,Pasadena,California,8867,6.3


In [8]:
df1 = pd.concat([dfs[0], dfs[2]], axis = 0, ignore_index = True)
df1.columns = ['Rank', 'City', 'State', 'Chinese-Americans', 'Percentage']
df1.drop(columns = 'Rank', inplace=True)
df1.sort_values(by='Chinese-Americans', ascending=False, inplace=True)
df1.reset_index(drop=True, inplace = True)
df1.head(10)

,City,State,Chinese-Americans,Percentage
0,New York,New York,562205,6.6
1,San Francisco,California,180372,21.2
2,Los Angeles,California,77073,2.0
3,San Jose,California,75582,7.5
4,Chicago,Illinois,52917,1.9
5,Honolulu,Hawaii,51149,5.2
6,San Diego,California,43715,3.2
7,Fremont,California,40752,17.9
8,Philadelphia,Pennsylvania,35451,2.3
9,Oakland,California,34106,8.3


### Get the geographical coordinates of each city

In [9]:
list_lat = []   # create empty lists
list_long = []

for index, row in df1.iterrows(): # iterate over rows in dataframe
    City = row['City']
    State = row['State']       
    address = str(City)+','+str(State)

    geolocator = Nominatim(user_agent="To_explorer")
    location = geolocator.geocode(address)   
    lat = location.latitude
    long = location.longitude

    list_lat.append(lat)
    list_long.append(long)


# create new columns from lists    

df1['latitude'] = list_lat   
df1['longitude'] = list_long

df1.head(10)

,City,State,Chinese-Americans,Percentage,latitude,longitude
0,New York,New York,562205,6.6,40.712728,-74.006015
1,San Francisco,California,180372,21.2,37.779281,-122.419236
2,Los Angeles,California,77073,2.0,34.053691,-118.242767
3,San Jose,California,75582,7.5,37.336191,-121.890583
4,Chicago,Illinois,52917,1.9,41.875562,-87.624421
5,Honolulu,Hawaii,51149,5.2,21.304547,-157.855676
6,San Diego,California,43715,3.2,32.717421,-117.162771
7,Fremont,California,40752,17.9,37.548270,-121.988572
8,Philadelphia,Pennsylvania,35451,2.3,39.952724,-75.163526
9,Oakland,California,34106,8.3,37.804456,-122.271356


### Define Foursquare Credentials and Version

In [10]:
# @hidden_cell

CLIENT_ID = 'WUVDZZ24YDZEJXMHUG25K30MFPDWZE5BAZN5NY2NYAH5DN5L' # my Foursquare ID
CLIENT_SECRET = 'TCTYJMHSEECCBXHD2ZCA2ZKL0DYSFECEF3YVTGNA1LVRO3HH' # my Foursquare Secret
VERSION = '20180605' # Foursquare API version

### Get the total number of Chinese restaurants in each city

In [11]:
search_query = 'Chinese food'

def getNumber(names, latitudes, longitudes, radius=5000, LIMIT=100):
    
    list_number=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            search_query,
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['venues']
        
        # count the total number of restaurants
        dataframe = json_normalize(results)
        number = len(dataframe.index)
        list_number.append(number)
    
    return(list_number)

In [12]:
Chinese_restaurant_Number = getNumber(names=df1['City'],
                                   latitudes=df1['latitude'],
                                   longitudes=df1['longitude']
                                  )

New York
San Francisco
Los Angeles
San Jose
Chicago
Honolulu
San Diego
Fremont
Philadelphia
Oakland
Houston
Irvine
Seattle
Boston
Fairfax
Sacramento
El Monte
Daly City
Sunnyvale
Bellevue
Austin
Plano
Portland
West Covina
Spring Valley
Elk Grove
Berkeley
Columbus
Pasadena
Jersey City
Jersey City
Ann Arbor
Torrance
Cambridge
Hayward
Madison
Pittsburgh
Naperville
Washington
Stockton
Enterprise
Rancho Cucamonga
Richmond
Fullerton
Cary
Chandler
Anaheim
Pomona
Arlington
Raleigh
Richardson
Long Beach
Riverside
Tempe
College Station
Durham
Durham
Huntington Beach
Thousand Oaks
Glendale
Concord
Henderson
Kent
Lexington
Salt Lake City
Garden Grove
Columbia
Orange
Providence
New Haven
Eugene
Overland Park
Boulder
Gainesville
Syracuse
Antioch
Worcester
Garland
Baton Rouge
Frisco
Miramar
Athens
Corona
Stamford
Vancouver
Norman
Norwalk
Carlsbad
Fort Collins
Roseville
Simi Valley
Carrollton
Costa Mesa
Rochester
Burbank
Vallejo
Fairfield
Denton
Pearland


In [13]:
df1['number'] = Chinese_restaurant_Number
df1.head()

,City,State,Chinese-Americans,Percentage,latitude,longitude,number
0,New York,New York,562205,6.6,40.712728,-74.006015,50
1,San Francisco,California,180372,21.2,37.779281,-122.419236,50
2,Los Angeles,California,77073,2.0,34.053691,-118.242767,50
3,San Jose,California,75582,7.5,37.336191,-121.890583,50
4,Chicago,Illinois,52917,1.9,41.875562,-87.624421,50


In [14]:
df2 = df1.sort_values(by='number', ascending=True, inplace=False)
df2.head(10)

,City,State,Chinese-Americans,Percentage,latitude,longitude,number
16,El Monte,California,20190,17.5,36.701463,-118.755997,0
40,Enterprise,Nevada,5879,4.5,36.002834,-115.201299,12
90,Simi Valley,California,1606,1.3,34.269447,-118.781482,14
98,Pearland,Texas,1112,1.1,29.563976,-95.286430,18
58,Thousand Oaks,California,3889,3.0,34.171427,-118.910588,22
85,Norman,Oklahoma,1751,1.5,35.222572,-97.439482,25
61,Henderson,Nevada,3329,1.2,36.039146,-114.981923,25
79,Frisco,Texas,2077,1.5,33.150674,-96.823612,26
25,Elk Grove,California,10758,6.6,38.408799,-121.371618,27
75,Antioch,California,2342,2.2,38.004921,-121.805789,27


Chose the city that has relatively small number of Chinese restaurants but still a large Chinese-American population

In [15]:
df3 = df2[(df2['number'] < 50) & (df2.index < 50)]
df3

,City,State,Chinese-Americans,Percentage,latitude,longitude,number
16,El Monte,California,20190,17.5,36.701463,-118.755997,0
40,Enterprise,Nevada,5879,4.5,36.002834,-115.201299,12
25,Elk Grove,California,10758,6.6,38.408799,-121.371618,27
44,Cary,North Carolina,5283,3.4,35.788305,-78.781196,32
14,Fairfax,Virginia,28806,2.5,38.846224,-77.306373,34
42,Richmond,California,5523,5.1,37.935758,-122.347749,38
31,Ann Arbor,Michigan,7998,6.8,42.268157,-83.731229,40
37,Naperville,Illinois,6584,4.5,41.772870,-88.147928,42


To choose between these cities, we consider the AreaVibes Livability Score https://www.areavibes.com

In [16]:
df3['livability score']=[63, 75, 75, 86, 76, 57, 79, 75]
df3

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,City,State,Chinese-Americans,Percentage,latitude,longitude,number,livability score
16,El Monte,California,20190,17.5,36.701463,-118.755997,0,63
40,Enterprise,Nevada,5879,4.5,36.002834,-115.201299,12,75
25,Elk Grove,California,10758,6.6,38.408799,-121.371618,27,75
44,Cary,North Carolina,5283,3.4,35.788305,-78.781196,32,86
14,Fairfax,Virginia,28806,2.5,38.846224,-77.306373,34,76
42,Richmond,California,5523,5.1,37.935758,-122.347749,38,57
31,Ann Arbor,Michigan,7998,6.8,42.268157,-83.731229,40,79
37,Naperville,Illinois,6584,4.5,41.772870,-88.147928,42,75


### Considering the livability score, we would choose 'Cary, North Carolina' as the best place to open Chinese restaurant. 

## 2. Explore its neighborhoood

There are some information about neighborhoods in Cary, North Carolina in the website http://www.city-data.com/city/Cary-North-Carolina.html , but the names of the neighborhoods are not separated. I'm trying to separate the neighborhood names and put them in a list called "mylist".

In [17]:
import re
ChangeWords = [' neighborhood ']
neighborhood_name = "Adams Park neighborhood Allenbrook neighborhood Ambience Place neighborhood Andover neighborhood Applecross Townhomes neighborhood Appleton Acres neighborhood Ardmore at Reedy Creek neighborhood Arlington Park at Amberly neighborhood Ashcroft neighborhood Ashley Meadows neighborhood Ashley Woods neighborhood Bali neighborhood Balmoral neighborhood Barrington Park neighborhood Battery at High House neighborhood Baybrook neighborhood Beacon Cove neighborhood Beaver Pond neighborhood Beechtree neighborhood Benedum Place neighborhood Bent Creek neighborhood Berkeley neighborhood Bexley at Weston neighborhood Bishop's Gate neighborhood Blackstone at Amberly neighborhood Bloomsbury at Cary Park neighborhood Bond Pointe neighborhood Bonnell Patio Homes neighborhood Boxwood neighborhood Briarcliff neighborhood Brighton Woods neighborhood Brookgreen neighborhood Brookline Village neighborhood Brookridge Estates neighborhood Brookstone neighborhood Brownfield Patio Homes neighborhood Buckherst West neighborhood Buckhurst neighborhood Bud El Acres neighborhood Buddys Ponderosa neighborhood Callan Park at Waldo neighborhood Cambridge neighborhood Camden Forest neighborhood Camden Park Townhomes neighborhood Cameron Pond neighborhood Candle Wood neighborhood Canterbury neighborhood Carolina Preserve at Amberly neighborhood Carpenter Village neighborhood Carr Hills neighborhood Carramore neighborhood Carrington Estates neighborhood Carrousel Park neighborhood Cary Park neighborhood Cary Pines neighborhood Carystone Park Townhomes neighborhood Castlebrook neighborhood Catherwood Place neighborhood Cedar Creek neighborhood Center Court Cluster Homes neighborhood Charleston Woods neighborhood Chatham Woods neighborhood Chesney Glen neighborhood Cheverly neighborhood Chimney Rise Townhomes neighborhood Churchill Downs neighborhood Claridge Place neighborhood Clubhomes neighborhood Coles Creek at Weatherstone neighborhood Colonial Townes neighborhood Concannon Townhomes neighborhood Cornwall Village neighborhood Coronado Village neighborhood Cotswold neighborhood Courtney Pines neighborhood Courtyards of Preston neighborhood Coventry neighborhood Coventry Woods neighborhood Creekview Bungalows neighborhood Crown Pointe neighborhood Dail neighborhood Danbury neighborhood Davidson Point Townhomes neighborhood Devereaux neighborhood Devintage neighborhood Dogwood Estates neighborhood Downing Place neighborhood Drawbridge neighborhood Dublin Woods neighborhood Dutchess Village neighborhood Edinborough Pointe neighborhood Evans Estates neighborhood Evergreen neighborhood Excalibur neighborhood Fairfax of The Parkway neighborhood Fairview Townhomes neighborhood Fairways of Lochmere neighborhood Fairwinds of Lochmere neighborhood Falconers Ridge neighborhood Farmington Woods neighborhood Farren Forest neighborhood Fenton Estate neighborhood Fernwood neighborhood Fieldstone Village neighborhood Fishers Creek neighborhood Forest Creek neighborhood Forest Park neighborhood Fox Chase neighborhood Foxwerth neighborhood Franklin Heights neighborhood Gabel Farms neighborhood Gallup neighborhood Georgetowne neighborhood Giverny neighborhood Gleneagles neighborhood Glenkirk neighborhood Glenmitt Stone neighborhood Glenridge neighborhood Glover Acres neighborhood Goodwood neighborhood Governor's Row neighborhood Greenwood Acres neighborhood Greenwood Forest neighborhood Hadley Place neighborhood Hampstead Park neighborhood Hampton Park Townhomes neighborhood Hanover Place neighborhood Harbour Towne neighborhood Harmony Glen (Harmony Pointe) neighborhood Harrison Place neighborhood Harvest Ridge neighborhood Hazelwood neighborhood Hearthstone Farms neighborhood Hemlock Ridge neighborhood Heritage Pines neighborhood Hermitage Park neighborhood Hickory Ridge neighborhood Highcroft neighborhood Highland Oaks neighborhood Highland Park neighborhood Highlands West neighborhood Highlands on The Bluffs neighborhood Highlands on The Lake neighborhood Hilliard Forest neighborhood Hillsdale Forest neighborhood Holloway neighborhood Homestead neighborhood Hortons Creek neighborhood Hunt at Green Level Crossing neighborhood Hunter Creek Townhomes neighborhood Hunter Park neighborhood Hunter Street neighborhood Huntsmoor neighborhood Indian Wells neighborhood Iron Gate neighborhood Ivy Creek neighborhood Ivy Meadows neighborhood Jason Court neighborhood Kenilworth neighborhood Kensington at Regency neighborhood Kilarney Pointe neighborhood Kildaire Farms neighborhood Kings Wood neighborhood Kingsmill neighborhood Kistler Court neighborhood Krendle Woods neighborhood Kurtsbrook neighborhood Lake Wellingborough neighborhood Lakepoint Village neighborhood Lakepointe at Lochmere neighborhood Lakeridge Townhomes neighborhood Lakeside at Chimney Rise neighborhood Landsdowne neighborhood Legacy at Carpenter Village neighborhood Lexington Park at Amberly neighborhood Linville Ridge neighborhood Linwood neighborhood Lions Gate at Regency neighborhood Lochaven neighborhood Lochbain neighborhood Lochmere neighborhood Lochridge Cluster neighborhood Lochside neighborhood Lochview South neighborhood Lochwinds neighborhood Macarthur Park neighborhood Macdonald Woods neighborhood Macgregor neighborhood Madison Oaks neighborhood Madison Place neighborhood Magnolia Estates neighborhood Magnolia Woods neighborhood Martin neighborhood Matchbox Village neighborhood Maynard Forest neighborhood Maynard Oaks neighborhood Maynard Summit Townhomes neighborhood Maynard Woods neighborhood Meadow Oaks neighborhood Meadowmont neighborhood Medcon Court neighborhood Melbourne neighborhood Montclair neighborhood Muir Woods neighborhood Normandale neighborhood Normandie neighborhood North Meadow neighborhood Northwoods neighborhood Oak Ridge Acres neighborhood Oakwood Heights neighborhood Olde Carpenter Townhomes neighborhood Olde Salem neighborhood Oxford Hunt Club neighborhood Page Wood Forest neighborhood Park Crest at Laura Duncan neighborhood Park Grove neighborhood Park Place Townhomes neighborhood Park Village neighborhood Parkcanyon neighborhood Parkcrest neighborhood Parkgate neighborhood Parkknoll neighborhood Parkmeadow neighborhood Parkroyale neighborhood Paxton Townhomes neighborhood Pebble Creek neighborhood Peninsula at Preston neighborhood Pepperdine neighborhood Picardy Pointe neighborhood Picardy Village neighborhood Pine Valley neighborhood Pink Acres neighborhood Pirates Cove neighborhood Plantation Estates neighborhood Planters Woods Townhomes neighborhood Pointe View Townes neighborhood Port High Meadows neighborhood Preston neighborhood Prestwick Place neighborhood Providence neighborhood Queensferry neighborhood Rainbow Estates neighborhood Raquet Club at Wimbledon neighborhood Red Fox neighborhood Regency Park Estates neighborhood Renaissance at Regency neighborhood Ridgefield Farms neighborhood Ridgepath neighborhood Ridges of Northwoods neighborhood Riggsbee Farm neighborhood Riverwalk neighborhood Rockhampton neighborhood Rosewall Townhomes at Wimbledon neighborhood Royal Oaks neighborhood Royal Ridge neighborhood Russell Hills neighborhood Samuels Keep Townhomes neighborhood Sandler at Amberly neighborhood Saratoga Orchard Park neighborhood Saratoga Park neighborhood Savon Heights neighborhood Scott Farm neighborhood Scottish Hills neighborhood Shennandoah neighborhood Sherborne neighborhood Sherwood Greens neighborhood Silver Oaks neighborhood Silvercliff neighborhood Silvercreek neighborhood Silvercrest neighborhood Silverlake neighborhood Silverridge neighborhood Somerset neighborhood Southbridge neighborhood Southchase neighborhood Southwick neighborhood Springbrook neighborhood Stanton Place neighborhood Sterling at Kildaire Farms neighborhood Stone Creek Village neighborhood Stonecrest neighborhood Stonehaven neighborhood Stonewater neighborhood Stoneybrook Estates neighborhood Storm Court neighborhood Strathmoor neighborhood Strathmore at Cary neighborhood Summer Pointe neighborhood Summer Ridge Townhomes neighborhood Summerwinds neighborhood Sunset Hills neighborhood Sunset Terrace neighborhood Sussex neighborhood Swallow Hill neighborhood Talton Place neighborhood Tanglewood neighborhood Tatton Place neighborhood Taylors Pond neighborhood Thamesford neighborhood The Arbor neighborhood The Bluffs neighborhood The Coves neighborhood The Enclave at Windemere neighborhood The Forest neighborhood The Greens neighborhood The Hermitage neighborhood The Highlands neighborhood The Lawns at Wimbledon neighborhood The Overlook neighborhood The Park of Kildaire neighborhood The Ranches neighborhood The Renaissance at Regency Estates neighborhood The Reserve neighborhood Thornewood neighborhood Timberridge neighborhood Towns of Madison neighborhood Trafalgar neighborhood Trappers Forest neighborhood Trappers Run neighborhood Triangle Forest neighborhood Tryon Villas neighborhood Twin Oaks Villa Townhomes neighborhood Upchurch Farms neighborhood Verde Park neighborhood Vicksburg neighborhood Village Green neighborhood Village Square at Amberly neighborhood Walnut Hills neighborhood Walnut Ridge neighborhood Washington Square at Amberly - Townhomes neighborhood Waterford Estates neighborhood Waterford Townhomes neighborhood Waterford Woods neighborhood Weatherfield Townhomes neighborhood Weathersby neighborhood Weatherstone neighborhood Wellington Place neighborhood Wessex neighborhood West Pines neighborhood West Street neighborhood Westchester Woods neighborhood Westfield neighborhood Weston Oaks neighborhood Westover Park Townhomes neighborhood Westview Cove neighborhood Weycroft neighborhood Wheatley neighborhood Whisperwood neighborhood White Oak Estates neighborhood Whitebridge neighborhood Whittington Park neighborhood Wickham Place neighborhood Willaimsburg Manor neighborhood Williamsburgh Commons neighborhood Willoughby Place neighborhood Windbrooke neighborhood Windchase at Beechtree neighborhood Windermere neighborhood Windsong neighborhood Windsor Oaks neighborhood Wishing Well Village neighborhood Wood Ridge neighborhood Woodcliff neighborhood Woodhaven neighborhood Woodland Acres neighborhood Woodlands at Green Level Crossing neighborhood Woods of Fairfax neighborhood Woodshire of Wellington neighborhood Woodwinds neighborhood Wordsmith Court neighborhood Wyndfall neighborhood Wynston Ridge"
big_regex = re.compile('|'.join(map(re.escape, ChangeWords)))
the_message = big_regex.sub('","', neighborhood_name)
the_message


'Adams Park","Allenbrook","Ambience Place","Andover","Applecross Townhomes","Appleton Acres","Ardmore at Reedy Creek","Arlington Park at Amberly","Ashcroft","Ashley Meadows","Ashley Woods","Bali","Balmoral","Barrington Park","Battery at High House","Baybrook","Beacon Cove","Beaver Pond","Beechtree","Benedum Place","Bent Creek","Berkeley","Bexley at Weston","Bishop\'s Gate","Blackstone at Amberly","Bloomsbury at Cary Park","Bond Pointe","Bonnell Patio Homes","Boxwood","Briarcliff","Brighton Woods","Brookgreen","Brookline Village","Brookridge Estates","Brookstone","Brownfield Patio Homes","Buckherst West","Buckhurst","Bud El Acres","Buddys Ponderosa","Callan Park at Waldo","Cambridge","Camden Forest","Camden Park Townhomes","Cameron Pond","Candle Wood","Canterbury","Carolina Preserve at Amberly","Carpenter Village","Carr Hills","Carramore","Carrington Estates","Carrousel Park","Cary Park","Cary Pines","Carystone Park Townhomes","Castlebrook","Catherwood Place","Cedar Creek","Center Court

In [18]:
mylist = ["Adams Park","Allenbrook","Ambience Place","Andover","Applecross Townhomes","Appleton Acres","Ardmore at Reedy Creek","Arlington Park at Amberly","Ashcroft","Ashley Meadows","Ashley Woods","Bali","Balmoral","Barrington Park","Battery at High House","Baybrook","Beacon Cove","Beaver Pond","Beechtree","Benedum Place","Bent Creek","Berkeley","Bexley at Weston","Bishop\'s Gate","Blackstone at Amberly","Bloomsbury at Cary Park","Bond Pointe","Bonnell Patio Homes","Boxwood","Briarcliff","Brighton Woods","Brookgreen","Brookline Village","Brookridge Estates","Brookstone","Brownfield Patio Homes","Buckherst West","Buckhurst","Bud El Acres","Buddys Ponderosa","Callan Park at Waldo","Cambridge","Camden Forest","Camden Park Townhomes","Cameron Pond","Candle Wood","Canterbury","Carolina Preserve at Amberly","Carpenter Village","Carr Hills","Carramore","Carrington Estates","Carrousel Park","Cary Park","Cary Pines","Carystone Park Townhomes","Castlebrook","Catherwood Place","Cedar Creek","Center Court Cluster Homes","Charleston Woods","Chatham Woods","Chesney Glen","Cheverly","Chimney Rise Townhomes","Churchill Downs","Claridge Place","Clubhomes","Coles Creek at Weatherstone","Colonial Townes","Concannon Townhomes","Cornwall Village","Coronado Village","Cotswold","Courtney Pines","Courtyards of Preston","Coventry","Coventry Woods","Creekview Bungalows","Crown Pointe","Dail","Danbury","Davidson Point Townhomes","Devereaux","Devintage","Dogwood Estates","Downing Place","Drawbridge","Dublin Woods","Dutchess Village","Edinborough Pointe","Evans Estates","Evergreen","Excalibur","Fairfax of The Parkway","Fairview Townhomes","Fairways of Lochmere","Fairwinds of Lochmere","Falconers Ridge","Farmington Woods","Farren Forest","Fenton Estate","Fernwood","Fieldstone Village","Fishers Creek","Forest Creek","Forest Park","Fox Chase","Foxwerth","Franklin Heights","Gabel Farms","Gallup","Georgetowne","Giverny","Gleneagles","Glenkirk","Glenmitt Stone","Glenridge","Glover Acres","Goodwood","Governor\'s Row","Greenwood Acres","Greenwood Forest","Hadley Place","Hampstead Park","Hampton Park Townhomes","Hanover Place","Harbour Towne","Harmony Glen (Harmony Pointe)","Harrison Place","Harvest Ridge","Hazelwood","Hearthstone Farms","Hemlock Ridge","Heritage Pines","Hermitage Park","Hickory Ridge","Highcroft","Highland Oaks","Highland Park","Highlands West","Highlands on The Bluffs","Highlands on The Lake","Hilliard Forest","Hillsdale Forest","Holloway","Homestead","Hortons Creek","Hunt at Green Level Crossing","Hunter Creek Townhomes","Hunter Park","Hunter Street","Huntsmoor","Indian Wells","Iron Gate","Ivy Creek","Ivy Meadows","Jason Court","Kenilworth","Kensington at Regency","Kilarney Pointe","Kildaire Farms","Kings Wood","Kingsmill","Kistler Court","Krendle Woods","Kurtsbrook","Lake Wellingborough","Lakepoint Village","Lakepointe at Lochmere","Lakeridge Townhomes","Lakeside at Chimney Rise","Landsdowne","Legacy at Carpenter Village","Lexington Park at Amberly","Linville Ridge","Linwood","Lions Gate at Regency","Lochaven","Lochbain","Lochmere","Lochridge Cluster","Lochside","Lochview South","Lochwinds","Macarthur Park","Macdonald Woods","Macgregor","Madison Oaks","Madison Place","Magnolia Estates","Magnolia Woods","Martin","Matchbox Village","Maynard Forest","Maynard Oaks","Maynard Summit Townhomes","Maynard Woods","Meadow Oaks","Meadowmont","Medcon Court","Melbourne","Montclair","Muir Woods","Normandale","Normandie","North Meadow","Northwoods","Oak Ridge Acres","Oakwood Heights","Olde Carpenter Townhomes","Olde Salem","Oxford Hunt Club","Page Wood Forest","Park Crest at Laura Duncan","Park Grove","Park Place Townhomes","Park Village","Parkcanyon","Parkcrest","Parkgate","Parkknoll","Parkmeadow","Parkroyale","Paxton Townhomes","Pebble Creek","Peninsula at Preston","Pepperdine","Picardy Pointe","Picardy Village","Pine Valley","Pink Acres","Pirates Cove","Plantation Estates","Planters Woods Townhomes","Pointe View Townes","Port High Meadows","Preston","Prestwick Place","Providence","Queensferry","Rainbow Estates","Raquet Club at Wimbledon","Red Fox","Regency Park Estates","Renaissance at Regency","Ridgefield Farms","Ridgepath","Ridges of Northwoods","Riggsbee Farm","Riverwalk","Rockhampton","Rosewall Townhomes at Wimbledon","Royal Oaks","Royal Ridge","Russell Hills","Samuels Keep Townhomes","Sandler at Amberly","Saratoga Orchard Park","Saratoga Park","Savon Heights","Scott Farm","Scottish Hills","Shennandoah","Sherborne","Sherwood Greens","Silver Oaks","Silvercliff","Silvercreek","Silvercrest","Silverlake","Silverridge","Somerset","Southbridge","Southchase","Southwick","Springbrook","Stanton Place","Sterling at Kildaire Farms","Stone Creek Village","Stonecrest","Stonehaven","Stonewater","Stoneybrook Estates","Storm Court","Strathmoor","Strathmore at Cary","Summer Pointe","Summer Ridge Townhomes","Summerwinds","Sunset Hills","Sunset Terrace","Sussex","Swallow Hill","Talton Place","Tanglewood","Tatton Place","Taylors Pond","Thamesford","The Arbor","The Bluffs","The Coves","The Enclave at Windemere","The Forest","The Greens","The Hermitage","The Highlands","The Lawns at Wimbledon","The Overlook","The Park of Kildaire","The Ranches","The Renaissance at Regency Estates","The Reserve","Thornewood","Timberridge","Towns of Madison","Trafalgar","Trappers Forest","Trappers Run","Triangle Forest","Tryon Villas","Twin Oaks Villa Townhomes","Upchurch Farms","Verde Park","Vicksburg","Village Green","Village Square at Amberly","Walnut Hills","Walnut Ridge","Washington Square at Amberly - Townhomes","Waterford Estates","Waterford Townhomes","Waterford Woods","Weatherfield Townhomes","Weathersby","Weatherstone","Wellington Place","Wessex","West Pines","West Street","Westchester Woods","Westfield","Weston Oaks","Westover Park Townhomes","Westview Cove","Weycroft","Wheatley","Whisperwood","White Oak Estates","Whitebridge","Whittington Park","Wickham Place","Willaimsburg Manor","Williamsburgh Commons","Willoughby Place","Windbrooke","Windchase at Beechtree","Windermere","Windsong","Windsor Oaks","Wishing Well Village","Wood Ridge","Woodcliff","Woodhaven","Woodland Acres","Woodlands at Green Level Crossing","Woods of Fairfax","Woodshire of Wellington","Woodwinds","Wordsmith Court","Wyndfall","Wynston Ridge"]
neigh = pd.DataFrame(mylist)
neigh.columns = ['Neighborhood']
neigh.head(10)

,Neighborhood
0,Adams Park
1,Allenbrook
2,Ambience Place
3,Andover
4,Applecross Townhomes
5,Appleton Acres
6,Ardmore at Reedy Creek
7,Arlington Park at Amberly
8,Ashcroft
9,Ashley Meadows


### Get the latitude and longitude values of each neighborhood

In [20]:
list_lat = []   # create empty lists
list_long = []

for index, row in neigh.iterrows(): # iterate over rows in dataframe
    City = 'Cary, North Carolina'
    Neighborhood = row['Neighborhood']       
    address = str(Neighborhood)+', '+str(City)

    geolocator = Nominatim(user_agent="To_explorer")
    location = geolocator.geocode(address) 
    if location: 
        lat = location.latitude 
        long = location.longitude
    else: 
        lat = None 
        long = None
    
    list_lat.append(lat)
    list_long.append(long)


# create new columns from lists    

neigh['latitude'] = list_lat   
neigh['longitude'] = list_long

neigh.head(10)

,Neighborhood,latitude,longitude
0,Adams Park,NaN,NaN
1,Allenbrook,35.775758,-78.771380
2,Ambience Place,35.739008,-78.773885
3,Andover,35.775421,-78.817316
4,Applecross Townhomes,35.757609,-78.814131
5,Appleton Acres,35.812176,-78.796356
6,Ardmore at Reedy Creek,35.800821,-78.765561
7,Arlington Park at Amberly,35.824618,-78.906770
8,Ashcroft,35.788305,-78.781196
9,Ashley Meadows,NaN,NaN


remove the neighborhoods that cannot find a location in geocode

In [22]:
neigh=neigh.dropna().reset_index(drop=True)
neigh.head(10)

,Neighborhood,latitude,longitude
0,Allenbrook,35.775758,-78.771380
1,Ambience Place,35.739008,-78.773885
2,Andover,35.775421,-78.817316
3,Applecross Townhomes,35.757609,-78.814131
4,Appleton Acres,35.812176,-78.796356
5,Ardmore at Reedy Creek,35.800821,-78.765561
6,Arlington Park at Amberly,35.824618,-78.906770
7,Ashcroft,35.788305,-78.781196
8,Ashley Woods,35.784135,-78.875776
9,Balmoral,35.766613,-78.805773


In [23]:
print('The dataframe has {} neighborhoods.'.format(neigh.shape[0]))

The dataframe has 275 neighborhoods.


### Create a map of Cary with neighborhoods superimposed on top.

In [24]:
# create map of Cary using latitude and longitude values
map_Cary = folium.Map(location=[35.788305, -78.781196], zoom_start=12)

# add markers to map
for lat, lng, neighborhood in zip(neigh['latitude'], neigh['longitude'], neigh['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Cary)
    
map_Cary

### We can now start to use Foursquare to explore the neighborhoods

First define a function to get the information of top venues nearby

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    LIMIT = 100
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Apply the function to the neighborhoods of Cary

In [26]:
Cary_venues = getNearbyVenues(names=neigh['Neighborhood'],
                                   latitudes=neigh['latitude'],
                                   longitudes=neigh['longitude']
                                  )


Allenbrook
Ambience Place
Andover
Applecross Townhomes
Appleton Acres
Ardmore at Reedy Creek
Arlington Park at Amberly
Ashcroft
Ashley Woods
Balmoral
Battery at High House
Baybrook
Beaver Pond
Beechtree
Benedum Place
Berkeley
Bexley at Weston
Blackstone at Amberly
Bond Pointe
Bonnell Patio Homes
Boxwood
Briarcliff
Brighton Woods
Brookgreen
Brookridge Estates
Brookstone
Buckhurst
Cambridge
Camden Forest
Camden Park Townhomes
Cameron Pond
Candle Wood
Canterbury
Carpenter Village
Carr Hills
Carramore
Cary Park
Cary Pines
Castlebrook
Catherwood Place
Cedar Creek
Center Court Cluster Homes
Charleston Woods
Chesney Glen
Cheverly
Chimney Rise Townhomes
Churchill Downs
Claridge Place
Coles Creek at Weatherstone
Colonial Townes
Concannon Townhomes
Cornwall Village
Coronado Village
Cotswold
Courtney Pines
Courtyards of Preston
Coventry
Coventry Woods
Crown Pointe
Danbury
Devereaux
Dogwood Estates
Downing Place
Drawbridge
Dublin Woods
Dutchess Village
Edinborough Pointe
Evans Estates
Evergreen
Ex

In [27]:
print(Cary_venues.shape)
Cary_venues.head(10)

(1532, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Allenbrook,35.775758,-78.77138,DQ Grill & Chill,35.776055,-78.768110,Fast Food Restaurant
1,Allenbrook,35.775758,-78.77138,Your Pie,35.775563,-78.767042,Pizza Place
2,Allenbrook,35.775758,-78.77138,Penn Station East Coast Subs,35.775265,-78.766391,Sandwich Place
3,Allenbrook,35.775758,-78.77138,Banana Leaf,35.776748,-78.767983,Chinese Restaurant
4,Allenbrook,35.775758,-78.77138,Primo Pizza NY Style,35.775195,-78.766781,Pizza Place
5,Allenbrook,35.775758,-78.77138,Ollie's Bargain Outlet,35.776453,-78.767318,Discount Store
6,Allenbrook,35.775758,-78.77138,ABC Store,35.776610,-78.766414,Liquor Store
7,Allenbrook,35.775758,-78.77138,Wing Stop,35.775282,-78.766721,Wings Joint
8,Allenbrook,35.775758,-78.77138,Dunham Park,35.777489,-78.770283,Park
9,Allenbrook,35.775758,-78.77138,Mac's Tavern,35.777119,-78.767848,Sports Bar


In [28]:
Cary_venues.groupby('Neighborhood').count().head(10)

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allenbrook,19,19,19,19,19,19
Ambience Place,21,21,21,21,21,21
Andover,6,6,6,6,6,6
Applecross Townhomes,2,2,2,2,2,2
Appleton Acres,4,4,4,4,4,4
Ardmore at Reedy Creek,7,7,7,7,7,7
Arlington Park at Amberly,5,5,5,5,5,5
Ashcroft,48,48,48,48,48,48
Ashley Woods,18,18,18,18,18,18


In [29]:
print('There are {} unique categories.'.format(len(Cary_venues['Venue Category'].unique())))

There are 205 unique categories.


## 3. Analyze each neighborhood

In [30]:
# one hot encoding
Cary_onehot=[]
Cary_onehot = pd.get_dummies(Cary_venues[['Venue Category']], prefix="", prefix_sep="")

# insert neighborhood column back to dataframe
Cary_onehot = Cary_onehot.drop(['Neighborhood'], axis=1)
Cary_onehot.insert(0, 'Neighborhood', Cary_venues['Neighborhood'], True) 

Cary_onehot.head(10)

,Neighborhood,ATM,Accessories Store,Acupuncturist,American Restaurant,Amphitheater,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Assisted Living,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Chinese Restaurant,Chiropractor,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Discount Store,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Electronics Store,Ethiopian Restaurant,Eye Doctor,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Fishing Spot,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Himalayan Restaurant,History Museum,Home Service,Hot Dog Joint,Hotel,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Lingerie Store,Liquor Store,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Moving Target,Multiplex,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightlife Spot,Noodle House,North Indian Restaurant,Optical Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Print Shop,Professional & Other Places,Pub,Racetrack,Real Estate Office,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Ski Area,Smoke Shop,Smoothie Shop,Soccer Field,Social Club,South American Restaurant,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Tanning Salon,Tennis Court,Thai Restaurant,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Weight Loss Center,Whisky Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Allenbrook,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Allenbrook,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Allenbrook,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [31]:
Cary_onehot.shape

(1532, 205)

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [32]:
Cary_grouped = Cary_onehot.groupby('Neighborhood').mean().reset_index()
Cary_grouped.head(10)

,Neighborhood,ATM,Accessories Store,Acupuncturist,American Restaurant,Amphitheater,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Assisted Living,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Chinese Restaurant,Chiropractor,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Discount Store,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Electronics Store,Ethiopian Restaurant,Eye Doctor,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Fishing Spot,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Himalayan Restaurant,History Museum,Home Service,Hot Dog Joint,Hotel,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Lingerie Store,Liquor Store,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Moving Target,Multiplex,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightlife Spot,Noodle House,North Indian Restaurant,Optical Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Print Shop,Professional & Other Places,Pub,Racetrack,Real Estate Office,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Ski Area,Smoke Shop,Smoothie Shop,Soccer Field,Social Club,South American Restaurant,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Tanning Salon,Tennis Court,Thai Restaurant,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Weight Loss Center,Whisky Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Allenbrook,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.105263,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.052632,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.105263,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.052632,0.000000,0.000000,0.0,0.0,0.0,0.052632,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.052632,0.0,0.0,0.052632,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.052632,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.052632,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.052632,0.000000,0.0,0.000000,0.0,0.0,0.0,0.157895,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.052632,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.052632,0.0,0.0,0.0,0.000000,0.000000

In [33]:
Cary_grouped.shape

(251, 205)

### Let's get the top 10 most common venues for each neighborhood and sort the venues in descending order

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [52]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Cary_grouped['Neighborhood']

for ind in np.arange(Cary_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Cary_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allenbrook,Pizza Place,Chinese Restaurant,Baseball Field,Cosmetics Shop,Fast Food Restaurant,Sports Bar,Liquor Store,Sandwich Place,Food,Grocery Store
1,Ambience Place,Gourmet Shop,Yoga Studio,Grocery Store,Café,Fast Food Restaurant,Burger Joint,Shopping Mall,Multiplex,Mexican Restaurant,Mediterranean Restaurant
2,Andover,Lake,Pool,Home Service,Performing Arts Venue,Trail,Harbor / Marina,Farm,Ethiopian Restaurant,Eye Doctor,Fabric Shop
3,Applecross Townhomes,Pool,Park,Yoga Studio,Ethiopian Restaurant,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market
4,Appleton Acres,Concert Hall,Pool,Fast Food Restaurant,Yoga Studio,Ethiopian Restaurant,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Farmers Market
5,Ardmore at Reedy Creek,Indian Restaurant,Supermarket,Pizza Place,Discount Store,Chinese Restaurant,Music Store,Ethiopian Restaurant,Yoga Studio,Fabric Shop,Food & Drink Shop
6,Arlington Park at Amberly,Athletics & Sports,Pizza Place,Bubble Tea Shop,Pharmacy,Grocery Store,Eye Doctor,Food & Drink Shop,Food,Flower Shop,Fishing Spot
7,Ashcroft,Food Truck,Brewery,Bakery,Pakistani Restaurant,Bridal Shop,Bus Station,Restaurant,Café,Chinese Restaurant,Pub
8,Ashley Woods,Chinese Restaurant,Mediterranean Restaurant,Ice Cream Shop,Jewelry Store,Pizza Place,Pharmacy,Donut Shop,Shoe Store,Fast Food Restaurant,South Indian Restaurant
9,Balmoral,Pizza Place,Hardware Store,Smoke Shop,Outdoors & Recreation,Yoga Studio,Electronics Store,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market


## Cluster Neighborhoods

Run k-means to cluster the neighborhood into 6 clusters.

In [53]:
# set number of clusters
kclusters = 10

Cary_grouped_clustering = Cary_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Cary_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 4, 3, 0, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [54]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Cary_merged = neigh

# merge Cary_grouped with previous neighborhood data to add latitude/longitude for each neighborhood
Cary_merged = Cary_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Cary_merged.head(10)

,Neighborhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allenbrook,35.775758,-78.771380,0.0,Pizza Place,Chinese Restaurant,Baseball Field,Cosmetics Shop,Fast Food Restaurant,Sports Bar,Liquor Store,Sandwich Place,Food,Grocery Store
1,Ambience Place,35.739008,-78.773885,0.0,Gourmet Shop,Yoga Studio,Grocery Store,Café,Fast Food Restaurant,Burger Joint,Shopping Mall,Multiplex,Mexican Restaurant,Mediterranean Restaurant
2,Andover,35.775421,-78.817316,2.0,Lake,Pool,Home Service,Performing Arts Venue,Trail,Harbor / Marina,Farm,Ethiopian Restaurant,Eye Doctor,Fabric Shop
3,Applecross Townhomes,35.757609,-78.814131,4.0,Pool,Park,Yoga Studio,Ethiopian Restaurant,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market
4,Appleton Acres,35.812176,-78.796356,3.0,Concert Hall,Pool,Fast Food Restaurant,Yoga Studio,Ethiopian Restaurant,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Farmers Market
5,Ardmore at Reedy Creek,35.800821,-78.765561,0.0,Indian Restaurant,Supermarket,Pizza Place,Discount Store,Chinese Restaurant,Music Store,Ethiopian Restaurant,Yoga Studio,Fabric Shop,Food & Drink Shop
6,Arlington Park at Amberly,35.824618,-78.906770,0.0,Athletics & Sports,Pizza Place,Bubble Tea Shop,Pharmacy,Grocery Store,Eye Doctor,Food & Drink Shop,Food,Flower Shop,Fishing Spot
7,Ashcroft,35.788305,-78.781196,0.0,Food Truck,Brewery,Bakery,Pakistani Restaurant,Bridal Shop,Bus Station,Restaurant,Café,Chinese Restaurant,Pub
8,Ashley Woods,35.784135,-78.875776,0.0,Chinese Restaurant,Mediterranean Restaurant,Ice Cream Shop,Jewelry Store,Pizza Place,Pharmacy,Donut Shop,Shoe Store,Fast Food Restaurant,South Indian Restaurant
9,Balmoral,35.766613,-78.805773,0.0,Pizza Place,Hardware Store,Smoke Shop,Outdoors & Recreation,Yoga Studio,Electronics Store,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market


Let's visualize the resulting clusters

In [55]:
Cary_merged=Cary_merged.dropna().reset_index(drop=True)
Cary_merged.head(10)

,Neighborhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allenbrook,35.775758,-78.771380,0.0,Pizza Place,Chinese Restaurant,Baseball Field,Cosmetics Shop,Fast Food Restaurant,Sports Bar,Liquor Store,Sandwich Place,Food,Grocery Store
1,Ambience Place,35.739008,-78.773885,0.0,Gourmet Shop,Yoga Studio,Grocery Store,Café,Fast Food Restaurant,Burger Joint,Shopping Mall,Multiplex,Mexican Restaurant,Mediterranean Restaurant
2,Andover,35.775421,-78.817316,2.0,Lake,Pool,Home Service,Performing Arts Venue,Trail,Harbor / Marina,Farm,Ethiopian Restaurant,Eye Doctor,Fabric Shop
3,Applecross Townhomes,35.757609,-78.814131,4.0,Pool,Park,Yoga Studio,Ethiopian Restaurant,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market
4,Appleton Acres,35.812176,-78.796356,3.0,Concert Hall,Pool,Fast Food Restaurant,Yoga Studio,Ethiopian Restaurant,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Farmers Market
5,Ardmore at Reedy Creek,35.800821,-78.765561,0.0,Indian Restaurant,Supermarket,Pizza Place,Discount Store,Chinese Restaurant,Music Store,Ethiopian Restaurant,Yoga Studio,Fabric Shop,Food & Drink Shop
6,Arlington Park at Amberly,35.824618,-78.906770,0.0,Athletics & Sports,Pizza Place,Bubble Tea Shop,Pharmacy,Grocery Store,Eye Doctor,Food & Drink Shop,Food,Flower Shop,Fishing Spot
7,Ashcroft,35.788305,-78.781196,0.0,Food Truck,Brewery,Bakery,Pakistani Restaurant,Bridal Shop,Bus Station,Restaurant,Café,Chinese Restaurant,Pub
8,Ashley Woods,35.784135,-78.875776,0.0,Chinese Restaurant,Mediterranean Restaurant,Ice Cream Shop,Jewelry Store,Pizza Place,Pharmacy,Donut Shop,Shoe Store,Fast Food Restaurant,South Indian Restaurant
9,Balmoral,35.766613,-78.805773,0.0,Pizza Place,Hardware Store,Smoke Shop,Outdoors & Recreation,Yoga Studio,Electronics Store,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market


In [56]:
# create map
latitude = 35.788305 
longitude = -78.781196
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Cary_merged['latitude'], Cary_merged['longitude'], Cary_merged['Neighborhood'], Cary_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster. 

### Cluster 1

In [57]:
Cary_merged.loc[Cary_merged['Cluster Labels'] == 0, Cary_merged.columns[[0] + list(range(4, Cary_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allenbrook,Pizza Place,Chinese Restaurant,Baseball Field,Cosmetics Shop,Fast Food Restaurant,Sports Bar,Liquor Store,Sandwich Place,Food,Grocery Store
1,Ambience Place,Gourmet Shop,Yoga Studio,Grocery Store,Café,Fast Food Restaurant,Burger Joint,Shopping Mall,Multiplex,Mexican Restaurant,Mediterranean Restaurant
5,Ardmore at Reedy Creek,Indian Restaurant,Supermarket,Pizza Place,Discount Store,Chinese Restaurant,Music Store,Ethiopian Restaurant,Yoga Studio,Fabric Shop,Food & Drink Shop
6,Arlington Park at Amberly,Athletics & Sports,Pizza Place,Bubble Tea Shop,Pharmacy,Grocery Store,Eye Doctor,Food & Drink Shop,Food,Flower Shop,Fishing Spot
7,Ashcroft,Food Truck,Brewery,Bakery,Pakistani Restaurant,Bridal Shop,Bus Station,Restaurant,Café,Chinese Restaurant,Pub
8,Ashley Woods,Chinese Restaurant,Mediterranean Restaurant,Ice Cream Shop,Jewelry Store,Pizza Place,Pharmacy,Donut Shop,Shoe Store,Fast Food Restaurant,South Indian Restaurant
9,Balmoral,Pizza Place,Hardware Store,Smoke Shop,Outdoors & Recreation,Yoga Studio,Electronics Store,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market
10,Battery at High House,Supermarket,Home Service,Gym / Fitness Center,Pharmacy,Pet Store,Chinese Restaurant,Salon / Barbershop,Business Service,Bus Station,Fast Food Restaurant
11,Baybrook,Bookstore,Social Club,Yoga Studio,Convenience Store,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market
16,Bexley at Weston,Café,Yoga Studio,Ethiopian Restaurant,Food Court,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market


### Cluster 2

In [58]:
Cary_merged.loc[Cary_merged['Cluster Labels'] == 1, Cary_merged.columns[[0] + list(range(4, Cary_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
54,Coventry Woods,Breakfast Spot,Yoga Studio,Eye Doctor,Food Court,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market
75,Franklin Heights,Insurance Office,Breakfast Spot,Yoga Studio,Eye Doctor,Food Court,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant
90,Hazelwood,Construction & Landscaping,Insurance Office,Ethiopian Restaurant,Food Court,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market
93,Hermitage Park,Construction & Landscaping,Insurance Office,Ethiopian Restaurant,Food Court,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market
176,Royal Ridge,Construction & Landscaping,Ethiopian Restaurant,Food Court,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market,Farm


### Cluster 3

In [59]:
Cary_merged.loc[Cary_merged['Cluster Labels'] == 2, Cary_merged.columns[[0] + list(range(4, Cary_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Andover,Lake,Pool,Home Service,Performing Arts Venue,Trail,Harbor / Marina,Farm,Ethiopian Restaurant,Eye Doctor,Fabric Shop
13,Beechtree,Pool,Playground,Trail,Yoga Studio,Electronics Store,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market
19,Bonnell Patio Homes,Construction & Landscaping,Trail,Pool,IT Services,Home Service,History Museum,Food,Flower Shop,Fishing Spot,Fast Food Restaurant
40,Chesney Glen,Pool,Health Food Store,Playground,Farm,Trail,Yoga Studio,Electronics Store,Food,Flower Shop,Fishing Spot
67,Falconers Ridge,Construction & Landscaping,IT Services,Arcade,Trail,Cafeteria,Ethiopian Restaurant,Food & Drink Shop,Food,Flower Shop,Fishing Spot
79,Glenkirk,Arts & Entertainment,Trail,Automotive Shop,Yoga Studio,Eye Doctor,Food Court,Food & Drink Shop,Food,Flower Shop,Fishing Spot
88,Harrison Place,Pool,Trail,Yoga Studio,Electronics Store,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market
101,Homestead,Pool,Trail,Yoga Studio,Electronics Store,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market
113,Kingsmill,Lake,Construction & Landscaping,Pool,Trail,Performing Arts Venue,Yoga Studio,Food,Flower Shop,Fishing Spot,Fast Food Restaurant
124,Lions Gate at Regency,Construction & Landscaping,IT Services,Arcade,Trail,Cafeteria,Ethiopian Restaurant,Food & Drink Shop,Food,Flower Shop,Fishing Spot


### Cluster 4

In [60]:
Cary_merged.loc[Cary_merged['Cluster Labels'] == 3, Cary_merged.columns[[0] + list(range(4, Cary_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Appleton Acres,Concert Hall,Pool,Fast Food Restaurant,Yoga Studio,Ethiopian Restaurant,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Farmers Market
43,Churchill Downs,Concert Hall,Yoga Studio,Ethiopian Restaurant,Food Court,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market
55,Crown Pointe,Concert Hall,Yoga Studio,Ethiopian Restaurant,Food Court,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market
71,Fishers Creek,Concert Hall,Pool,Fast Food Restaurant,Yoga Studio,Ethiopian Restaurant,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Farmers Market
143,Melbourne,Concert Hall,Yoga Studio,Ethiopian Restaurant,Food Court,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market
185,Silver Oaks,Concert Hall,Pool,Yoga Studio,Ethiopian Restaurant,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market
187,Silvercreek,Concert Hall,Yoga Studio,Ethiopian Restaurant,Food Court,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market
195,Springbrook,Concert Hall,Yoga Studio,Ethiopian Restaurant,Food Court,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market


### Cluster 5

In [61]:
Cary_merged.loc[Cary_merged['Cluster Labels'] == 4, Cary_merged.columns[[0] + list(range(4, Cary_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Applecross Townhomes,Pool,Park,Yoga Studio,Ethiopian Restaurant,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market
14,Benedum Place,Auto Workshop,Pool,Bar,Yoga Studio,Food Court,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant
15,Berkeley,Pool,Yoga Studio,Electronics Store,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market,Farm
18,Bond Pointe,Gym,Pool,Yoga Studio,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market,Farm
24,Brookstone,Pool,Pizza Place,Yoga Studio,Ethiopian Restaurant,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market
34,Cary Park,Pool,Supermarket,Pharmacy,Yoga Studio,Electronics Store,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market
36,Catherwood Place,Auto Workshop,Pool,Bar,Yoga Studio,Food Court,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant
38,Center Court Cluster Homes,Pool,IT Services,Trail,Electronics Store,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market
39,Charleston Woods,Gym,Home Service,Pool,Yoga Studio,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market,Farm
46,Colonial Townes,Gym / Fitness Center,Pool,Electronics Store,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market,Farm


### Cluster 6

In [62]:
Cary_merged.loc[Cary_merged['Cluster Labels'] == 5, Cary_merged.columns[[0] + list(range(4, Cary_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Beaver Pond,Park,Yoga Studio,Ethiopian Restaurant,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market,Farm
33,Carramore,IT Services,Park,Food Stand,Food Court,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market
35,Cary Pines,Park,Soccer Field,Trail,Yoga Studio,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market,Farm
49,Coronado Village,Scenic Lookout,Park,Yoga Studio,Ethiopian Restaurant,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market
73,Forest Park,Park,Yoga Studio,Ethiopian Restaurant,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market,Farm
84,Greenwood Acres,Spa,Park,Yoga Studio,Convenience Store,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market
94,Highcroft,Karaoke Bar,Park,Yoga Studio,Eye Doctor,Food Court,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant
99,Hillsdale Forest,Park,Auto Workshop,Yoga Studio,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant
131,Macdonald Woods,Park,Trail,Yoga Studio,Ethiopian Restaurant,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market
135,Magnolia Estates,Playground,Park,Yoga Studio,Ethiopian Restaurant,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market


### Cluster 7

In [63]:
Cary_merged.loc[Cary_merged['Cluster Labels'] == 6, Cary_merged.columns[[0] + list(range(4, Cary_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
77,Giverny,Arcade,Yoga Studio,Eye Doctor,Food Stand,Food Court,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant
133,Madison Oaks,Gym / Fitness Center,Gym,Yoga Studio,Food Stand,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market
161,Picardy Pointe,Gym / Fitness Center,Tennis Court,Food Stand,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market,Farm
180,Savon Heights,Print Shop,Gym / Fitness Center,Brewery,Arcade,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market
202,Strathmore at Cary,Print Shop,Gym / Fitness Center,Brewery,Arcade,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market
233,Westfield,Print Shop,Gym / Fitness Center,Brewery,Arcade,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market


### Cluster 8

In [64]:
Cary_merged.loc[Cary_merged['Cluster Labels'] == 7, Cary_merged.columns[[0] + list(range(4, Cary_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
58,Downing Place,Yoga Studio,Athletics & Sports,Deli / Bodega,Department Store,Food Stand,Food Court,Food & Drink Shop,Food,Flower Shop,Fishing Spot
59,Dublin Woods,Athletics & Sports,Bus Station,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market
65,Fairview Townhomes,Yoga Studio,Dog Run,Athletics & Sports,Department Store,Fabric Shop,Food Stand,Food Court,Food & Drink Shop,Food,Flower Shop
92,Heritage Pines,Athletics & Sports,Fried Chicken Joint,Food Stand,Food Court,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market
114,Kistler Court,Athletics & Sports,Fried Chicken Joint,Food Stand,Food Court,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market
174,Rosewall Townhomes at Wimbledon,Yoga Studio,Dog Run,Athletics & Sports,Department Store,Fabric Shop,Food Stand,Food Court,Food & Drink Shop,Food,Flower Shop


### Cluster 9

In [65]:
Cary_merged.loc[Cary_merged['Cluster Labels'] == 8, Cary_merged.columns[[0] + list(range(4, Cary_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Blackstone at Amberly,Lake,Bagel Shop,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market
28,Cameron Pond,Playground,Yoga Studio,Electronics Store,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market,Farm
41,Cheverly,Lake,Playground,Yoga Studio,Food Court,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market
42,Chimney Rise Townhomes,Lake,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market,Farm
118,Lakeridge Townhomes,Lake,Resort,Playground,Martial Arts Dojo,Yoga Studio,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant
146,Normandale,Lake,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market,Farm
206,Sussex,Lake,Food Truck,Food Court,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market,Farm


### Cluster 10

In [66]:
Cary_merged.loc[Cary_merged['Cluster Labels'] == 9, Cary_merged.columns[[0] + list(range(4, Cary_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Briarcliff,Resort,Yoga Studio,Electronics Store,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market,Farm
51,Courtney Pines,Resort,Yoga Studio,Electronics Store,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market,Farm
139,Maynard Woods,Resort,Yoga Studio,Electronics Store,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market,Farm
244,Wishing Well Village,Resort,Yoga Studio,Electronics Store,Food & Drink Shop,Food,Flower Shop,Fishing Spot,Fast Food Restaurant,Farmers Market,Farm
